<a href="https://colab.research.google.com/github/marcomag416/MLDL/blob/main/VerticalFlip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download GTA 5 dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from zipfile import ZipFile

gta_path = "/content/drive/MyDrive/MLDL-Proj/GTA5.zip"

gta_dataset_path = "./dataset/"

#extract zip file
with ZipFile(gta_path, 'r') as zip_ref:
    zip_ref.extractall(gta_dataset_path)


#download cityscapes dataset

In [3]:
with ZipFile("/content/drive/MyDrive/MLDL-Proj/Cityscapes.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset")

cityscape_dataset_path = "./dataset/Cityscapes/Cityspaces"

# Creating semantic segmented dataset

In [4]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt


# Define paths
images_path = gta_dataset_path + 'GTA5/images'
labels_path = gta_dataset_path + 'GTA5/labels'

# Initialize lists to hold data
data = []

# Load images and corresponding masks
for image_filename in os.listdir(images_path):
    if image_filename.endswith('.png'):
        image_path = os.path.join(images_path, image_filename)
        mask_path = os.path.join(labels_path, image_filename)

        # Check if corresponding mask file exists
        if os.path.exists(mask_path):
            # Open image and mask to ensure they can be loaded (optional, for validation)
            try:
                image = Image.open(image_path)
                mask = Image.open(mask_path)

                # Add data to list
                data.append({
                    'image_path': image_path,
                    'mask_path': mask_path
                })
            except Exception as e:
                print(f"Error loading {image_path} or {mask_path}: {e}")

# Create a DataFrame from the data list
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('gta5_segmentation_dataset.csv', index=False)

print("Semantic segmentation dataset created and saved as 'gta5_segmentation_dataset.csv'")

Semantic segmentation dataset created and saved as 'gta5_segmentation_dataset.csv'


# Wandb

In [5]:
#install and import wandb for data collecting
!pip install wandb
import wandb

wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Download pytorchdl_gta5

One of these 2 ways should work

2. upload the zip file manually

In [6]:
import zipfile

# Specify the path to the zip file
file_path = "/content/drive/MyDrive/MLDL-Proj/pytorchdl_gta5.zip"

# Specify the destination directory for unzipping
destination_path = "/content/pytorchdl_gta5"

# Extract the contents of the zip file
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_path)

# Transformation and preparing train loader with GTA dataset

In [7]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from pytorchdl_gta5.labels import GTA5Labels_TaskCV2017
import albumentations as A
from albumentations.pytorch import ToTensorV2

if __name__ != '__main__':
    raise Exception("This script should not be imported; it should be run directly.")

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define the custom dataset class
class GTASegmentationDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform
        self.label_mapping = self._create_label_mapping()
        self.color_jitter = T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
        self.gaussian_blur = T.GaussianBlur(kernel_size=(3, 7), sigma=(0.1, 5))

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.data_frame.iloc[idx, 0]
        mask_name = self.data_frame.iloc[idx, 1]

        image = Image.open(img_name).convert('RGB')
        mask = Image.open(mask_name).convert('RGB')

        # image = self.color_jitter(image)

        # image = self.gaussian_blur(image)

        if self.transform:
            augmented = self.transform(image=np.array(image), mask=np.array(mask))
            image, mask = augmented['image'], augmented['mask']

        mask = self._map_mask(np.array(mask))

        # Convert mask to tensor without normalization
        mask = torch.from_numpy(mask).long()  # Ensure the mask is of type long for cross-entropy loss

        return image, mask

    def _create_label_mapping(self):
        label_mapping = {label.color: label.ID for label in GTA5Labels_TaskCV2017.list_}
        label_mapping[(0, 0, 0)] = 255  # Ensure unmapped colors go to 'unlabeled'
        return label_mapping

    def _map_mask(self, mask):
        new_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.uint8)
        for color, label_id in self.label_mapping.items():
            color_mask = np.all(mask == color, axis=-1)
            new_mask[color_mask] = label_id  # Use label_id instead of color
        return new_mask

# Define paths
csv_file = 'gta5_segmentation_dataset.csv'

# Define image transformations
transform = A.Compose([
    A.Resize(720, 1280),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    # A.RandomRotate90(),
    # A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    # A.RandomResizedCrop(height=720, width=1280, scale=(0.8, 1.0)),
    ToTensorV2()
])

# Create the dataset and dataloader
train_dataset = GTASegmentationDataset(csv_file=csv_file, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Transformation and preparing val loader with cityscape dataset

In [8]:
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T
from torch.utils.data import DataLoader
import os


class Cityscapes(Dataset):
    def __init__(self, root_dir, split, transforms=None, label_type='gtFine_labelTrainIds'):
        self.root_dir = root_dir
        self.split = split
        self.transforms = transforms
        self.label_type = label_type

        self.images_dir = f"{root_dir}/images/{split}"
        self.labels_dir = f"{root_dir}/gtFine/{split}"

        self.image_paths = []
        self.label_paths = []

        # Manually iterate over directories
        cities = [city for city in os.listdir(self.images_dir) if os.path.isdir(f"{self.images_dir}/{city}")]
        for city in cities:
            img_dir_city = f"{self.images_dir}/{city}"
            lbl_dir_city = f"{self.labels_dir}/{city}"

            if not os.path.isdir(img_dir_city) or not os.path.isdir(lbl_dir_city):
                continue

            for img_file in os.listdir(img_dir_city):
                if img_file.endswith('_leftImg8bit.png'):
                    img_path = f"{img_dir_city}/{img_file}"
                    lbl_file = img_file.replace('_leftImg8bit.png', f'_{self.label_type}.png')
                    lbl_path = f"{lbl_dir_city}/{lbl_file}"

                    if os.path.isfile(img_path) and os.path.isfile(lbl_path):
                        self.image_paths.append(img_path)
                        self.label_paths.append(lbl_path)
                    else:
                        print(f"Warning: Image or label file not found for {img_file}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        lbl_path = self.label_paths[idx]

        image = Image.open(img_path).convert('RGB')
        label = Image.open(lbl_path)

        image = np.array(image)
        label = np.array(label)

        if self.transforms:
            augmented = self.transforms(image=image, mask=label)
            image, label = augmented['image'], augmented['mask']

        return image, label


# Example usage
image_transforms = A.Compose([
    A.Resize(512, 1024),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_dataset = Cityscapes(root_dir=cityscape_dataset_path, split='val', transforms=image_transforms)

val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

print(f"Val_Dataset size: {len(val_dataset)}")

Val_Dataset size: 500


# Download biseNet model

In [9]:
import sys
import requests
from zipfile import ZipFile
from io import BytesIO
model_url = "https://github.com/ooooverflow/BiSeNet/archive/refs/heads/master.zip"

# Send a GET request to the URL
response = requests.get(model_url)
# Check if the request was successful
if response.status_code == 200:
    #print(response.content)
    # Open the downloaded bytes and extract them
    with ZipFile(BytesIO(response.content)) as zip_file:
        zip_file.extractall('./')
    print('Download and extraction complete!')

sys.path.insert(0, './BiSeNet-master')

Download and extraction complete!


In [10]:
from model.build_BiSeNet import BiSeNet
from torch import nn

# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

context_path = 'resnet18'

#save hyperparameters
config = {
    "learning_rate": 1e-3,
    "max_epochs": 50,
    "batch_size": 8,
    "weight_decay": "None",
    "dataset": "Cityscapes",
    "scheduler": "None",
    "optimizer": "Adam",
    "transformations": "verticalFlip"
}

# Initialize the model
model = BiSeNet(num_classes=19, context_path=context_path).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

scheduler = None

# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 58.9MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:02<00:00, 73.6MB/s]


# Train and evaluation func

In [11]:
def train(model, optimizer_train, dataloader, loss_fn_train):
    model.train()  # Set the model to training mode
    train_loss = 0.0
    total = 0

    for idx, (inputs_train, targets_train) in enumerate(dataloader):
        inputs_train = inputs_train.to(device)
        targets_train = targets_train.to(device, dtype=torch.long)  # Move data to the appropriate device

        optimizer_train.zero_grad()  # Zero out gradients from the previous iteration
        outputs_train, cx1_sup, cx2_sup = model(inputs_train)  # Forward pass
        # print(outputs_train.shape, targets_train.shape)
        loss = loss_fn_train(outputs_train, targets_train)  # Calculate the loss

        aux_loss1 = loss_fn_train(cx1_sup, targets_train)
        aux_loss2 = loss_fn_train(cx2_sup, targets_train)

        loss = loss + aux_loss1 + aux_loss2

        loss.backward()  # Backward pass
        optimizer_train.step()  # Update the weights

        wandb.log({"train/Batch loss": loss})

        train_loss += loss.item() * inputs_train.size(0)  # Accumulate the total loss
        _, predicted_train = outputs_train.max(1)
        total += targets_train.size(0)

    # Calculate average loss for the epoch
    avg_loss = train_loss / total

    wandb.log({"train/Epoch loss": avg_loss})

    return avg_loss


def compute_iou(pred, target, num_classes):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(num_classes):
        pred_inds = (pred == cls)
        target_inds = (target == cls)
        intersection = (pred_inds[target_inds]).sum().item()
        union = pred_inds.sum().item() + target_inds.sum().item() - intersection
        if union == 0:
            ious.append(float('nan'))  # If there is no union, set IoU to NaN
        else:
            ious.append(intersection / union)

    return np.array(ious)

def eval(model, dataloader, loss_fn, device, num_classes=19):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    total = 0
    all_ious = []  # List to store IoUs for each batch

    with torch.no_grad():  # Disable gradient calculation during inference
        for inputs_test, targets_test in dataloader:
            inputs_test, targets_test = inputs_test.to(device), targets_test.to(device, dtype=torch.long)

            outputs_test = model(inputs_test)  # Forward pass
            loss = loss_fn(outputs_test, targets_test)  # Calculate the loss

            test_loss += loss.item() * inputs_test.size(0)  # Accumulate the total loss
            _, predicted_test = outputs_test.max(1)
            total += targets_test.size(0)

            # Compute IoU for this batch
            batch_ious = compute_iou(predicted_test, targets_test, num_classes)
            all_ious.append(batch_ious)

    # Calculate average loss
    avg_loss = test_loss / total

    wandb.log({})

    # Calculate mean IoU
    all_ious = np.array(all_ious)
    mean_iou = np.nanmean(all_ious, axis=0)  # Mean IoU for each class
    miou = np.nanmean(mean_iou)  # Mean IoU across all classes

    wandb.log({"val/Validation loss": avg_loss, "val/mIoU": miou})

    return avg_loss, miou, mean_iou

# Training

In [12]:
from timeit import default_timer as timer

#run names
project_name = "verticalFlip"
run_name = "vertical_flip"

#start wandb run
wandb.init(project=project_name, name=run_name, config=config)

start_time = timer()

# Setup training and save the results
for epoch in range(config["max_epochs"]):
    wandb.log({"Epoch": epoch+1})
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou, mean_iou = eval(model, val_dataloader, loss_fn, device=device)
    if scheduler != None:
      scheduler.step()
    #save model state every 5 epochs
    if((epoch + 1) % 5 == 0):
      torch.save(model.state_dict(), f"/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch{epoch}.pth")
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%, mean_iou: {mean_iou}")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")
wandb.finish()

KeyboardInterrupt: 

In [15]:
from timeit import default_timer as timer

#run names
project_name = "verticalFlip"
run_name = "vertical_flip"

#start wandb run
wandb.init(project=project_name, name=run_name, config=config)

start_time = timer()

model.load_state_dict(torch.load("/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch14.pth"))


# Setup training and save the results
for epoch in range(10,50):
    wandb.log({"Epoch": epoch+1})
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou, mean_iou = eval(model, val_dataloader, loss_fn, device=device)
    if scheduler != None:
      scheduler.step()
    #save model state every 5 epochs
    if((epoch + 1) % 5 == 0):
      torch.save(model.state_dict(), f"/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch{epoch}.pth")
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%, mean_iou: {mean_iou}")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")
wandb.finish()

Epoch,▁
Epoch,11


Epoch: 11, Loss: 2.0772765035629273, mIoU: 10.20%, mean_iou: [0.48139725 0.00193739 0.39781746 0.00512331 0.         0.02163198
 0.03570962 0.00384668 0.38658766 0.04027494 0.48458289 0.06203858
 0.         0.0153562  0.0010559  0.00099344 0.         0.
 0.        ]
Epoch: 12, Loss: 2.079955993652344, mIoU: 11.64%, mean_iou: [1.92585468e-01 4.38711651e-02 4.69832381e-01 1.04891920e-02
 3.24405932e-04 8.98303649e-02 3.84058588e-02 2.57474682e-02
 5.44543399e-01 1.84992054e-02 5.65991737e-01 3.90610404e-02
 0.00000000e+00 1.69681869e-01 3.35401552e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Epoch: 13, Loss: 1.5896351318359374, mIoU: 12.86%, mean_iou: [0.36863147 0.01477917 0.48733744 0.00437004 0.0387619  0.09223042
 0.02197452 0.02152097 0.58332587 0.02997362 0.41660712 0.1103165
 0.         0.2434304  0.00334514 0.00709008 0.         0.
 0.        ]
Epoch: 14, Loss: 2.9551833527088167, mIoU: 10.30%, mean_iou: [0.09803327 0.0069354  0.34065721 0.00898345 0.01163934

KeyboardInterrupt: 

In [ ]:
from timeit import default_timer as timer

#run names
project_name = "verticalFlip"
run_name = "vertical_flip"

#start wandb run
wandb.init(project=project_name, name=run_name, config=config)

start_time = timer()

model.load_state_dict(torch.load("/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch14.pth"))


# Setup training and save the results
for epoch in range(15,50):
    wandb.log({"Epoch": epoch+1})
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou, mean_iou = eval(model, val_dataloader, loss_fn, device=device)
    if scheduler != None:
      scheduler.step()
    #save model state every 5 epochs
    if((epoch + 1) % 3 == 0):
      torch.save(model.state_dict(), f"/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch{epoch}.pth")
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%, mean_iou: {mean_iou}")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")
wandb.finish()

wandb: Currently logged in as: kharazmireyhaneh25 (mldlteam). Use `wandb login --relogin` to force relogin


Epoch: 16, Loss: 2.1721465182304383, mIoU: 10.35%, mean_iou: [3.10493002e-01 7.37963187e-04 3.13616064e-01 2.90941001e-04
 3.45094072e-03 5.55010654e-02 5.24908509e-02 4.72180719e-03
 6.49928854e-01 5.68260774e-02 3.34053815e-01 9.82836409e-02
 0.00000000e+00 7.83082971e-02 7.09115142e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Epoch: 17, Loss: 1.798123652458191, mIoU: 12.00%, mean_iou: [0.50544989 0.00373073 0.4881316  0.00323693 0.00753909 0.02728203
 0.02240413 0.00833072 0.40150603 0.04524649 0.53865087 0.09016016
 0.         0.13830326 0.0008626  0.         0.         0.
 0.        ]
Epoch: 18, Loss: 2.8770883054733276, mIoU: 8.57%, mean_iou: [9.16084078e-02 6.44375590e-03 2.76186499e-01 6.72360455e-03
 8.16482855e-03 4.33642500e-02 4.73962360e-02 5.37168461e-03
 4.69872424e-01 3.31140527e-02 4.96973238e-01 6.87852734e-02
 0.00000000e+00 6.77951362e-02 6.22295095e-03 4.92044399e-05
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Epoch: 19, Loss: 1.973437063217

In [ ]:
from timeit import default_timer as timer

#run names
project_name = "verticalFlip"
run_name = "vertical_flip"

#start wandb run
wandb.init(project=project_name, name=run_name, config=config)

start_time = timer()

model.load_state_dict(torch.load("/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch17.pth"))


# Setup training and save the results
for epoch in range(18,50):
    wandb.log({"Epoch": epoch+1})
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou, mean_iou = eval(model, val_dataloader, loss_fn, device=device)
    if scheduler != None:
      scheduler.step()
    #save model state every 5 epochs
    if((epoch + 1) % 3 == 0):
      torch.save(model.state_dict(), f"/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch{epoch}.pth")
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%, mean_iou: {mean_iou}")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")
wandb.finish()

wandb: Currently logged in as: kharazmireyhaneh25 (mldlteam). Use `wandb login --relogin` to force relogin


Epoch: 19, Loss: 2.1143917438983917, mIoU: 11.11%, mean_iou: [2.82396410e-01 5.66290406e-04 3.43405190e-01 1.26058268e-03
 5.88392807e-03 3.56434987e-02 3.54505374e-02 1.90034191e-02
 6.28264469e-01 6.11779257e-02 4.57299498e-01 1.41093678e-01
 0.00000000e+00 8.81453750e-02 9.39847342e-03 1.87638454e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Epoch: 20, Loss: 1.972057569503784, mIoU: 11.36%, mean_iou: [4.91648113e-01 3.15456130e-03 4.99096819e-01 2.62191595e-03
 9.21663094e-03 2.36768138e-02 2.72936812e-02 1.13172139e-02
 2.68373955e-01 3.02928097e-02 5.21895183e-01 1.08751304e-01
 1.58561639e-04 1.55371473e-01 3.48394929e-03 2.93068129e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Epoch: 21, Loss: 2.9250822310447693, mIoU: 8.67%, mean_iou: [0.1048724  0.00545567 0.33579375 0.00685055 0.01005392 0.0258795
 0.03775054 0.00690912 0.41452937 0.0304191  0.5013745  0.07043702
 0.         0.08997978 0.00662187 0.00065482 0.         0.
 0.        ]
Epoch: 22, Loss: 2.4477485897541

In [ ]:
from timeit import default_timer as timer

#run names
project_name = "verticalFlip"
run_name = "vertical_flip"

#start wandb run
wandb.init(project=project_name, name=run_name, config=config)

start_time = timer()

model.load_state_dict(torch.load("/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch29.pth"))


# Setup training and save the results
for epoch in range(30,50):
    wandb.log({"Epoch": epoch+1})
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou, mean_iou = eval(model, val_dataloader, loss_fn, device=device)
    if scheduler != None:
      scheduler.step()
    #save model state every 5 epochs
    if((epoch + 1) % 3 == 0):
      torch.save(model.state_dict(), f"/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch{epoch}.pth")
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%, mean_iou: {mean_iou}")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")
wandb.finish()

wandb: Currently logged in as: kharazmireyhaneh25 (mldlteam). Use `wandb login --relogin` to force relogin


Epoch: 31, Loss: 2.405568759918213, mIoU: 10.51%, mean_iou: [3.03692558e-01 6.75210943e-04 3.00369830e-01 3.85559061e-05
 4.62220954e-03 2.95671646e-02 2.96716481e-02 1.97090651e-03
 6.57153320e-01 5.63238908e-02 5.03497873e-01 9.28376641e-02
 0.00000000e+00 1.24331482e-02 1.50384051e-03 2.00834670e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Epoch: 32, Loss: 2.102115821361542, mIoU: 11.55%, mean_iou: [5.30985044e-01 9.11374963e-05 5.04210782e-01 4.48453118e-03
 2.39330161e-02 1.27735716e-02 1.20801868e-02 6.76891700e-03
 4.50376497e-01 3.97027899e-02 4.95437039e-01 9.46421117e-02
 0.00000000e+00 1.88978260e-02 3.72726941e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Epoch: 33, Loss: 2.4265887694358828, mIoU: 10.08%, mean_iou: [2.93115864e-01 2.06863818e-05 3.55897321e-01 8.59879704e-03
 1.23948645e-02 2.75497757e-02 4.19145207e-02 9.87604146e-03
 4.46973117e-01 3.02748378e-02 4.87526868e-01 8.51233586e-02
 0.00000000e+00 1.07227747e-01 7.19826472e-03 1.621210

In [ ]:
from timeit import default_timer as timer

#run names
project_name = "verticalFlip"
run_name = "vertical_flip"

#start wandb run
wandb.init(project=project_name, name=run_name, config=config)

start_time = timer()

model.load_state_dict(torch.load("/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch41.pth"))


# Setup training and save the results
for epoch in range(42,51):
    wandb.log({"Epoch": epoch+1})
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou, mean_iou = eval(model, val_dataloader, loss_fn, device=device)
    if scheduler != None:
      scheduler.step()
    #save model state every 5 epochs
    if((epoch + 1) % 5 == 0):
      torch.save(model.state_dict(), f"/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch{epoch}.pth")
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%, mean_iou: {mean_iou}")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")
wandb.finish()

Epoch,▁
train/Batch loss,▁
Epoch,43
train/Batch loss,0.77067


Epoch: 43, Loss: 1.7911332342624664, mIoU: 12.33%, mean_iou: [3.71640565e-01 2.04210606e-02 4.30048330e-01 8.72967673e-03
 1.60934500e-02 4.50811610e-02 2.79056765e-02 1.04943296e-02
 5.72526615e-01 5.54928714e-02 4.09262958e-01 1.31611277e-01
 0.00000000e+00 2.24484054e-01 1.29478064e-02 5.91112291e-03
 0.00000000e+00 1.91721467e-05 0.00000000e+00]
Epoch: 44, Loss: 1.8884821417331696, mIoU: 12.53%, mean_iou: [0.33695894 0.03341811 0.38998398 0.00881927 0.00958558 0.04917132
 0.04508304 0.0178866  0.6243244  0.06398652 0.45742782 0.0967789
 0.         0.22355016 0.01508341 0.00792871 0.         0.
 0.        ]
Epoch: 45, Loss: 1.7825755677223205, mIoU: 11.74%, mean_iou: [3.99431621e-01 6.65840904e-02 3.31160144e-01 7.08420236e-03
 4.93745885e-06 4.74847644e-02 1.80896949e-02 6.31878544e-03
 5.70698179e-01 6.31134113e-02 5.03877499e-01 1.12052800e-01
 0.00000000e+00 9.76113840e-02 6.27737155e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Epoch: 46, Loss: 2.12547434926

In [13]:
from timeit import default_timer as timer

#run names
project_name = "verticalFlip"
run_name = "vertical_flip"

#start wandb run
wandb.init(project=project_name, name=run_name, config=config)

start_time = timer()

model.load_state_dict(torch.load("/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch41.pth"))


# Setup training and save the results
for epoch in range(45,51):
    wandb.log({"Epoch": epoch+1})
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou, mean_iou = eval(model, val_dataloader, loss_fn, device=device)
    if scheduler != None:
      scheduler.step()
    #save model state every 5 epochs
    if((epoch + 1) % 2 == 0):
      torch.save(model.state_dict(), f"/content/drive/MyDrive/MLDL-Proj/weights/verticalflip_epoch{epoch}.pth")
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%, mean_iou: {mean_iou}")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")
wandb.finish()

Epoch,▁
train/Batch loss,█▄█▂▆▄▃▅▂▃▄▃▂▃▄▄▃▃▃▃▂▃▃▃▁▃▃▁▃▂▃▂▃▃▂▃▃▂▂▃
Epoch,46
train/Batch loss,0.52136


Epoch: 46, Loss: 1.8383326225280763, mIoU: 12.35%, mean_iou: [3.54754222e-01 3.79423335e-02 4.07174200e-01 7.27587959e-03
 2.06653869e-02 6.27822814e-02 3.59119858e-02 1.67599532e-02
 6.20949375e-01 7.34102824e-02 4.55890819e-01 1.14678176e-01
 0.00000000e+00 1.30555110e-01 7.99137999e-03 5.34343924e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Epoch: 47, Loss: 1.8075232791900635, mIoU: 13.31%, mean_iou: [2.72647158e-01 8.37287691e-02 4.25859449e-01 1.24737489e-02
 2.15915061e-02 7.63162424e-02 6.17734122e-02 2.88154881e-02
 6.33136432e-01 5.06908901e-02 4.98275497e-01 1.44367286e-01
 9.15645207e-04 1.98167003e-01 1.24542841e-02 6.91948458e-03
 0.00000000e+00 1.17340353e-04 4.98826291e-04]
Epoch: 48, Loss: 1.7618919403553008, mIoU: 13.40%, mean_iou: [0.34691868 0.03948454 0.41220506 0.02119697 0.0035863  0.09213554
 0.04913569 0.00898861 0.65201255 0.06390144 0.54864297 0.15035759
 0.         0.14973835 0.00473067 0.00213186 0.         0.
 0.        ]
Epoch: 49, Loss: 2.4505193614

KeyboardInterrupt: 